In [1]:
import pandas as pd
articles_df = pd.read_csv("bbc_preprocessed.csv")

In [2]:
articles_df.head()

,date,section,title,description,content
0,2017-01-01,BBC Trending,dead gorilla became meme bbc news,harambe shot sad incident cincinnati lived mil...,face launched thousand meme harambe gorilla ca...
1,2017-01-01,US & Canada,sign changed new year prank bbc news,prankster change worldfamous hollywood sign re...,iconic sign changed overnight new year eve res...
2,2017-01-01,Business,billion pound fail claim bbc news,huge payouts benefit compensation pension lott...,bumper lottery draw organised following team g...
3,2017-01-01,Health,psychological secret successful resolution bbc...,psychological trick help people achieve stick ...,important achievable goal excess festive seaso...
4,2017-01-02,UK Politics,next paul bbc news,tumultuous brexit victory ukips challenge year...,paul nuttall left hoping push labour hard ukip...


In [3]:
from transformers import pipeline, AutoTokenizer

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

/Users/chikro/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Tokenization-aware truncation function
def tokenize_and_truncate(text, max_length=512):
    tokens = tokenizer(text, truncation=True, max_length=max_length, return_tensors="pt")
    return tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)

# Extract the content of the first article
first_article_content = articles_df.loc[0, "content"]

# Truncate and summarize
truncated_content = tokenize_and_truncate(first_article_content, max_length=512)
summary = summarizer(truncated_content, max_length=150, min_length=40, do_sample=False)

# Display the result
print("Original Content:")
print(first_article_content)
print("\nTruncated Content:")
print(truncated_content)
print("\nSummary:")
print(summary[0]["summary_text"])

Original Content:
face launched thousand meme harambe gorilla capture collective online psyche sad story could even sadder may threeyearold child fell enclosure cincinnati zoo one western lowland gorilla inside started dragging boy around mommy right mommy love boy mother shouted bystander became increasingly panicked finally fearing boy life danger zoo worker killed harambe single shot boy escaped without serious injury event captured youtube video watched million time harambes death touched heated predictable debate zoo welfare standard whether lethal force necessary wasnt expected came next harambe became memeified image spread far wide throughout internet became subject serious unserious campaign even memorialised song join conversation story started spontaneous real outpouring shock grief killing would gone enclosure say frank paris one people used hashtag ripharambe express sadness quickly began spread hour gorilla death although lived state away los angeles paris along many othe

In [3]:
import os
from transformers import pipeline, AutoTokenizer
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Disable tokenizer parallelism to avoid the error
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# Tokenization-aware truncation function
def tokenize_and_truncate(text, max_length=512):
    tokens = tokenizer(text, truncation=True, max_length=max_length, return_tensors="pt")
    return tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)

# Summarization function for a single row
def summarize_content(row):
    text = tokenize_and_truncate(row.content, max_length=128)  # Use attribute access for 'content'
    summary = summarizer(text, max_length=128, min_length=40, do_sample=False)
    return summary[0]["summary_text"]

# Function to process rows in parallel using multiple CPU threads and tqdm for progress bar
def process_data_parallel(dataframe, num_workers=8):
    # Wrap executor.map with tqdm to show progress
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(tqdm(executor.map(summarize_content, dataframe.itertuples(index=False)), total=len(dataframe), desc="Summarizing"))
    return results

summaries = process_data_parallel(articles_df)
articles_df['summarised'] = summaries

# Display results
display(articles_df)

/Users/chikro/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Your max_length is set to 128, but your input_length is only 127. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=63)
Your max_length is set to 128, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)
Your max_length is set to 128, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer

KeyboardInterrupt: 